In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import os
import glob
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from pathlib import Path
from torch.optim.lr_scheduler import _LRScheduler
import json

from typing import List
import os
from functools import partial
import torch.nn.functional as F
from timeit import default_timer as timer
from utils import get_dataloaders

from models.utils import *
from datagen import kinematic_feature_names, kinematic_feature_names_jigsaws, class_names, all_class_names, state_variables

from tqdm import tqdm

Dataset & Dataloaders

In [4]:

### -------------------------- DATA -----------------------------------------------------
tasks = ["Suturing"]
Features = kinematic_feature_names_jigsaws[38:] + state_variables #kinematic features + state variable features

one_hot = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
observation_window = 10
prediction_window = 10
batch_size = 64

user_left_out = 2
cast = True
include_image_features = False
normalizer = '' # ('standardization', 'min-max', 'power', '')
step = 1 # 1 - 30 Hz

train_dataloader, valid_dataloader = get_dataloaders(tasks,
                                                     user_left_out,
                                                     observation_window,
                                                     prediction_window,
                                                     batch_size,
                                                     one_hot,
                                                     class_names = class_names['Suturing'],
                                                     feature_names = Features,
                                                     include_image_features=include_image_features,
                                                     cast = cast,
                                                     normalizer = normalizer,
                                                     step=step)

print("datasets lengths: ", len(train_dataloader.dataset), len(valid_dataloader.dataset))
print("X shape: ", train_dataloader.dataset.X.shape, valid_dataloader.dataset.X.shape)
print("Y shape: ", train_dataloader.dataset.Y.shape, valid_dataloader.dataset.Y.shape)

# loader generator aragement: (src, tgt, future_gesture, future_kinematics)
print("Obs Kinematics Shape: ", train_dataloader.dataset[0][0].shape) 
print("Obs Target Shape: ", train_dataloader.dataset[0][1].shape)
print("Future Target Shape: ", train_dataloader.dataset[0][2].shape)
print("Future Kinematics Shape: ", train_dataloader.dataset[0][3].shape)
print("Train N Trials: ", train_dataloader.dataset.get_num_trials())
print("Train Max Length: ", train_dataloader.dataset.get_max_len())
print("Test N Trials: ", valid_dataloader.dataset.get_num_trials())
print("Test Max Length: ", valid_dataloader.dataset.get_max_len())
print("Features: ", train_dataloader.dataset.get_feature_names())



[3155, 3266, 3048, 2449, 2527, 2653, 2002, 2303, 2642, 1758, 3583, 2925, 2516, 2869, 2482, 2803, 2374, 2235, 2372, 2028, 9011, 3378, 5160, 2620, 2453, 5274, 3320, 4020, 3279, 4310, 4411, 4145, 3730, 3700]
['G1' 'G10' 'G11' 'G2' 'G3' 'G4' 'G5' 'G6' 'G8' 'G9']
[0 0 0 ... 2 2 2]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
[5555, 3420, 3267, 3256, 2847]
['G1' 'G10' 'G11' 'G2' 'G3' 'G4' 'G5' 'G6' 'G8' 'G9']
[0 0 0 ... 2 2 2]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
datasets lengths:  11078 1832
X shape:  (110801, 43) (18345, 43)
Y shape:  (110801, 10) (18345, 10)
Obs Kinematics Shape:  (10, 43)
Obs Target Shape:  (11, 10)
Future Target Shape:  (10, 10)
Future Kinematics Shape:  (10, 43)
Train N Trials:  34
Train Max Length:  9011
Test N Trials:  5
Test Max Length:  55

In [5]:

# Example tensor of shape [batch_size, seq_len, features]
tensor = torch.tensor([[[1, 2, 3], [3, 2, 1], [1, 1, 2]],
                       [[2, 3, 3], [1, 2, 2], [3, 2, 3]]])
tensor = torch.zeros(64,10,14)
print(tensor.shape)
# Calculate the most common feature along the seq_len axis for each batch
most_common_feature = torch.mode(tensor, dim=1).values

print(most_common_feature.shape)


torch.Size([64, 10, 14])
torch.Size([64, 14])


In [6]:
def find_mostcommon(tensor,device):
    # # 64,10,10
    # # Count the occurrences of each row
    # unique_rows, counts = np.unique(batch_y, axis=0, return_counts=True)

    # # Get the index of the row with the highest count
    # most_common_row_index = np.argmax(counts)

    # # Get the most common row
    # most_common_row = unique_rows[most_common_row_index]
    
    # batch_y_bin = most_common_row
    
    # batch_y_bin = torch.from_numpy(batch_y_bin)
    
    batch_y_bin = torch.mode(tensor, dim=1).values
    batch_y_bin = batch_y_bin.to(device)
    
    return batch_y_bin
    

In [7]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):

        out, _ = self.lstm(x)

        out = self.fc(out)

        return out

In [8]:

class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]



def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler



Transformer+LSTM

In [9]:
# target output size of 5

class GlobalMaxPooling1D(nn.Module):

    def __init__(self, data_format='channels_last'):
        super(GlobalMaxPooling1D, self).__init__()
        self.data_format = data_format
        self.step_axis = 1 if self.data_format == 'channels_last' else 2

    def forward(self, input):
        return torch.max(input, axis=self.step_axis).values
    

m = GlobalMaxPooling1D()
input = torch.randn(1, 64, 8)
output = m(input)
print(output.shape)


class TransformerModel(nn.Module):
    
    def __init__(self, input_dim, output_dim, d_model, nhead, num_layers, hidden_dim, layer_dim, dropout=0.1):
        super().__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout), num_layers=num_layers
        )
        
        self.lstm = LSTMModel(d_model, hidden_dim, layer_dim, output_dim)
        
        self.max_pool = GlobalMaxPooling1D()
        self.fc = nn.Linear(input_dim, d_model)
        self.out = nn.Linear(d_model, output_dim)
        
    def forward(self, x):
        
        
        x = self.fc(x)
        
        
        x = self.max_pool(x)
        
        x = self.transformer(x)
    
        x = self.lstm(x)
        
        return x
    


torch.Size([1, 8])


In [10]:
def initiate_model(d_model, nhead, num_layers,input_dim,output_dim, hidden_dim, layer_dim, lr, iterations_per_epoch ):

    model = TransformerModel(input_dim=input_dim, output_dim=output_dim, d_model=d_model, nhead=nhead, num_layers=num_layers, hidden_dim=hidden_dim, layer_dim=layer_dim)

    
    
    model = model.cuda()
    
    optimizer = optim.Adam(model.parameters(), lr=lr) # adam
    
    # optimizer = torch.optim.RMSprop(model.parameters(), lr=lr) # custom
    
    sched = CyclicLR(optimizer, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

    criterion = nn.CrossEntropyLoss()
    
    return model, optimizer,sched, criterion


In [11]:
def eval_loop(model, test_dataloader, criterion):
    model.eval()
    with torch.no_grad():
        # eval
        losses = []
        ypreds, gts = [], []
        
        for src, tgt, future_gesture, future_kinematics in test_dataloader:
            src = src
            y = find_mostcommon(tgt[:,1:,:], device)
            
            y_pred = model(src) # [64,10]
        
            pred = torch.argmax(y_pred,dim=1)
            gt = torch.argmax(y,dim=1)
            
            pred = pred.cpu().numpy()
            gt = gt.cpu().numpy()
            
            ypreds.append(pred)
            gts.append(gt)
            
            loss = criterion(y_pred, y)

            losses.append(loss.item())
          
        ypreds = np.concatenate(ypreds)  
        gts = np.concatenate(gts)  
        
        get_classification_report(ypreds,gts,test_dataloader.dataset.get_target_names())
            
        # Compare each element and count matches
        matches = np.sum(ypreds == gts)

        # Calculate accuracy
        accuracy = matches / len(ypreds)

        print("Accuracy:", accuracy)
        
        return np.mean(losses), ypreds, gts


In [12]:


def traintest_loop(train_dataloader,test_dataloader,model,optimizer,scheduler,criterion, epochs):
    
    # training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for bi, (src, tgt, future_gesture, future_kinematics) in enumerate(tqdm(train_dataloader)):

            optimizer.zero_grad()
            
            src = src
            y = find_mostcommon(tgt[:,1:,:], device)
            
            y_pred = model(src) # [64,10]
            # print('prediction,gt:',y_pred.shape, y.shape, tgt.shape)
            
            loss = criterion(y_pred, y)
            loss.backward()

            optimizer.step()
            scheduler.step()
            
            running_loss += loss.item()
            
        print(f"Training Epoch {epoch+1}, Loss: {running_loss / len(train_dataloader):.6f}")
        
        # evaluation loop
        val_loss, ypreds, gts = eval_loop(model,test_dataloader,criterion)
        print(f"Valdiation Epoch {epoch+1}, Loss: {val_loss:.6f}")
        





In [27]:
        

lr = 0.00001

batch_size = 16
seq_len = 10

batch = next(iter(train_dataloader))
features = batch[0].shape[-1]
output_dim = batch[2].shape[-1]

input_dim = features  

# lstm
hidden_dim = 512
layer_dim = 2
seq_dim = 128

#transformer
d_model = 512
nhead=4
num_layers=4

task = "Suturing"

epochs = 10
# iterations_per_epoch = len(train_dataloader)
iterations_per_epoch = 500

batch = next(iter(train_dataloader))
features = batch[0].shape[-1]
output_dim = batch[1].shape[-1]

print(features, output_dim)

model,optimizer,scheduler,criterion = initiate_model(d_model, nhead, num_layers,input_dim,output_dim, hidden_dim, layer_dim, lr,  iterations_per_epoch=iterations_per_epoch)
    

43 10


In [28]:
subjects = [2,3,4,5,6,7,8,9]
subjects = [2]


with open("results.json", "w") as outfile:
    
    accuracy = []
    for subject in (subjects):

        model,optimizer,scheduler,criterion = initiate_model(d_model, nhead, num_layers,input_dim,output_dim, hidden_dim, layer_dim, lr,  iterations_per_epoch=iterations_per_epoch)

        user_left_out = subject

        train_dataloader, valid_dataloader = get_dataloaders(tasks,
                                                            user_left_out,
                                                            observation_window,
                                                            prediction_window,
                                                            batch_size,
                                                            one_hot,
                                                            class_names = class_names['Suturing'],
                                                            feature_names = Features,
                                                            include_image_features=include_image_features,
                                                            cast = cast,
                                                            normalizer = normalizer,
                                                            step=step)



        traintest_loop(train_dataloader,valid_dataloader,model,optimizer,scheduler,criterion, epochs)


        # Serializing json
    json_object = json.dumps(accuracy, indent=4)
    outfile.write(json_object)
        


[3155, 3266, 3048, 2449, 2527, 2653, 2002, 2303, 2642, 1758, 3583, 2925, 2516, 2869, 2482, 2803, 2374, 2235, 2372, 2028, 9011, 3378, 5160, 2620, 2453, 5274, 3320, 4020, 3279, 4310, 4411, 4145, 3730, 3700]
['G1' 'G10' 'G11' 'G2' 'G3' 'G4' 'G5' 'G6' 'G8' 'G9']
[0 0 0 ... 2 2 2]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
[5555, 3420, 3267, 3256, 2847]
['G1' 'G10' 'G11' 'G2' 'G3' 'G4' 'G5' 'G6' 'G8' 'G9']
[0 0 0 ... 2 2 2]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


100%|██████████| 693/693 [00:04<00:00, 171.37it/s]
/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior

Training Epoch 1, Loss: 1.810559
Accuracy: 0.5900655021834061
Valdiation Epoch 1, Loss: 1.360735


100%|██████████| 693/693 [00:03<00:00, 178.54it/s]
/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior

Training Epoch 2, Loss: 1.442408
Accuracy: 0.6441048034934498
Valdiation Epoch 2, Loss: 1.174507


...

100%|██████████| 693/693 [00:03<00:00, 176.76it/s]
/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior

Training Epoch 9, Loss: 0.756308
Accuracy: 0.6899563318777293
Valdiation Epoch 9, Loss: 1.004946


100%|██████████| 693/693 [00:03<00:00, 176.52it/s]


Training Epoch 10, Loss: 0.714397
Accuracy: 0.6768558951965066
Valdiation Epoch 10, Loss: 1.042300


/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/kesharaw/anaconda3/envs/pytorch/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

In [28]:
acc = []
for x in accuracy:
    acc.append(x['accuracy'])
    
print(np.average(acc))

0.6777466186830305


In [27]:
# 2. Create model save path 
MODEL_PATH = "./checkpoints/"
MODEL_NAME = "01_pytorch_workflow_model_0.pth"
# MODEL_NAME = str(d_model) + "_" + str(nhead) + "_" + str(num_layers) + "_" + MODEL_NAME
MODEL_NAME =  "transformer_encoder__lstm_" + MODEL_NAME
MODEL_SAVE_PATH = MODEL_PATH +"/"+ MODEL_NAME



# 3. Save the model state dict 
print(f"Saving model to: {MODEL_SAVE_PATH}")

torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH) 


print("done saving!")

Saving model to: ./checkpoints//transformer_encoder__lstm_01_pytorch_workflow_model_0.pth
done saving!


In [ ]:
def evaluation_loop():

    model.eval()

    total_accuracy = []
    for i, batch in enumerate(test_dataloader):
        x, y, y_seq = batch
        x = x.to(torch.float32)
        y = y.to(torch.float32)
        y_seq = y_seq.to(torch.float32)
        
        y_pred = model(x)
        
        total_inputs = 0
        true_pred = []
        
        
        for idx,y in enumerate(y_pred):
            
            total_inputs += 1
            
            output_argmax = torch.argmax(y)
            gt_argmax = torch.argmax(y_seq[idx])
        
            if(output_argmax == gt_argmax):
                true_pred.append(output_argmax)
                
            accuracy = len(true_pred)/total_inputs
            
            # print("Accuracy: ",accuracy)
            total_accuracy.append(accuracy)
            
            
                
        loss = criterion(y_pred, y_seq)
        # print(i, "Loss: ", loss)
        
    print("Average accuracy: ", np.average(total_accuracy))
    
    
evaluation_loop()